# Health Insurance Pricing & Factors of Influence using Python Data Analysis (Draft)
Notebook created by Mary T. <br><br>
This project's objective is to utilize different models with machine learning to predict customer health insurance charges. The initial algorithm of interest is linear regression, however this notebook will test other methods and scope algorithm accuracy.<br><br>
The data used within this project is [Medical Cost Personal Datasets](http://https://www.kaggle.com/mirichoi0218/insurance) and was originally obtained from [GitHub](http://https://github.com/stedy/Machine-Learning-with-R-datasets/blob/master/insurance.csv).<br><br>
Inspiration for this project came from the question: 'Can you accurately predict insurance costs?'

## Abstract
A new health insurance company is seeking a strategy to increase revenue, such as accruing more annual premiums or setting plan prices correctly to minimize loss. <br><br>
To do this, the company wants to identify the most at-risk customer groups and predict how much their average medical expenses will be. With this information, the insurance company will be able to properly price their plans in accordance to predicted medical charges. <br><br>
Poor pricing of plans or premiums will incur more loss to the insurance company and too much loss will cause the company to fail. For insurance, customers may utilize  too much of the benefits than the company can actually cover. <br><br>
To remain competitive an organization must provide the greatest return on the investment made in health insurance premiums to sustain enrollment. The strategy is to set proper annual premiums at an affordable rate to serve mutual interest: nurture enrollment and minimize loss. <br><br>
In a social sense, if insurance enrollment drops, health of the general population suffers as those most susceptible to disease and viral infections are the least likely to report skewing the information public health official need to direct investment (i.e. COVID) in social welfare. On the business side, incorrectly pricing plans or premiums and having clients utilize more benefits than companies can cover will lead to business loss and failure.

### Goals
Possible goals for this project:
* Utilize machine learning to predict medical charge pricing
* Discuss other factors that may affect medical charge pricing
* Create strategies for insurance businesses to use in order to increase revenue and maximise profit
* Predict annual premium pricing to maximize profits and minimize loss to realize a competitive advantage
* Predict how the prices for medical insurance vary according to customer groups
* Identify the most at risk customers from the data
* Observe how medical insurance prices affect people of different customer groups 
* Determine how medical companies benefit from data analytics

## Data Information
* **age**: insurance customer age in years
* **sex**: insurance customer gender: female, male
* **bwi**: body mass index
* **children**: # of children covered by health insurance or # of dependents
* **smoker**: smoking: yes, no
* **region**: the customer's residential area in the US: northeast, southeast, southwest, northwest
* **charges**: customer's medical costs billed by health insurance in dollars

## Importing Libraries
Here are the libraries utilized and needed for this notebook. 

In [ ]:
# Libraries to be imported
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Exploratory Data Analysis (EDA)
Exploratory data analysis is used to grasp a general understanding of the orginal dataset, such as how many rows or columns there are, what the datatypes are, whether there are null values, and similar.

In [ ]:
# definiing dataset variable and reading ine .csv file
df = pd.read_csv('../input/insurance/insurance.csv')

# previewing the first few rows of data
df.head()

In [ ]:
# seeing number of rows, columns, null counts and the data types
df.info()

In [ ]:
# specifically seeing data types
df.dtypes

In [ ]:
# summary statistics of the data and its numerical columns
df.describe()

In [ ]:
# seeing count of rows and columns
df.shape

In [ ]:
# determining whether there is any missing/null/NA data
df.isnull().sum()

## Visualizations with Original Data
The following visualizations will use the original dataset provided, meaning 'object' datatypes will still be present. Encoded dataset visualizations will follow after this section.

### Charges

In [ ]:
# general view of charges distribution
fig_cost = go.Figure()
fig_cost.add_trace(go.Histogram(x = df['charges']))
fig_cost.update_layout(title_text = 'Charges Distribution', xaxis_title_text = 'Medical Cost', yaxis_title_text = 'Count', bargap  = 0.05, template = 'seaborn', width = 600, height = 600)
fig_cost.show()

### Age

In [ ]:
# defining ages and creating age ranges
young = df[(df['age'] >= 18) & (df['age'] <= 39)]
middle = df[(df['age'] >= 40) & (df['age'] <= 59)]
old = df[df['age'] >= 60]

In [ ]:
# visualizing ages without ranges
fig_age = go.Figure()
fig_age.add_trace(go.Histogram(x = df['age']))
fig_age.update_layout(title_text = 'Age Distribution', xaxis_title_text = 'Age', yaxis_title_text = 'Count', bargap = 0.05, template = 'seaborn', width = 500, height = 500
)
fig_age.show()

In [ ]:
# visualizing ages with age ranges
fig_age = go.Figure()
fig_age.add_trace(go.Bar(x = ['Young Adults'], y = [len(young)], name = 'Young Adults'))
fig_age.add_trace(go.Bar(x = ['Middle Aged Adults'], y = [len(middle)], name = 'Middle Aged Adults'))
fig_age.add_trace(go.Bar(x = ['Old Adults'], y = [len(old)],  name = 'Old Adults'))
fig_age.update_layout(title_text = 'Total Count for Age Groups', xaxis_title_text = 'Age Group', yaxis_title_text = 'Count', bargap = 0.05, template = 'seaborn', width = 500, height = 500
)
fig_age.show()

In [ ]:
# visualizing with boxplots and points
fig_age = go.Figure()
fig_age.add_trace(go.Box(y = young['charges'], boxpoints = 'all', name = 'Young Adults'))
fig_age.add_trace(go.Box(y = middle['charges'], boxpoints = 'all', name = 'Middle Aged Adults'))
fig_age.add_trace(go.Box(y = old['charges'], boxpoints='all', name = 'Old Adults'))
fig_age.update_layout( title_text = 'Age Group and Charges', xaxis_title_text = 'Age Group', yaxis_title_text = 'Charges', template = 'seaborn', width = 600, height = 600
)
fig_age.show()

### Sex

In [ ]:
# defining male female
male = df[df['sex'] == 'male']
female = df[df['sex'] == 'female']

In [ ]:
# visualizing female and male costs barplot
fig_sex = go.Figure()
fig_sex.add_trace(go.Histogram(x = male['charges'], name = 'Male', marker_color = '#6495ED'))
fig_sex.add_trace(go.Histogram(x = female['charges'], name = 'Female', marker_color = '#A9A9A9'))
fig_sex.update_layout(barmode='stack', title_text = 'Sex', xaxis_title_text = 'Charges', yaxis_title='Count', template = 'seaborn', bargap = 0.1, width=500, height=600
)
fig_sex.show()

In [ ]:
# visualizing via pie chart
colors= ['#A9A9A9', '#6495ED']
labels = ['Male', 'Female']
values = [len(male), 
          len(female)]
fig_sex = go.Figure()
fig_sex.add_trace(go.Pie(labels = labels, values = values, hole = .4, marker_colors = colors))
fig_sex.update_layout( title_text = 'Total Count of Sex', template = 'seaborn', width = 500, height = 600
)
fig_sex.show()

In [ ]:
# visualizing boxplot of sex and medical charges
fig_sex = go.Figure()
fig_sex.add_trace(go.Box(y=male['charges'], boxpoints='all', marker_color = '#A9A9A9', name='Male'))
fig_sex.add_trace(go.Box(y=female['charges'], boxpoints='all', marker_color = '#6495ED', name='Female'))
fig_sex.update_layout( title_text = 'Sex and Charges', xaxis_title_text = 'Sex', yaxis_title='Charges', template = 'seaborn', width=600, height=600)
fig_sex.show()

### BMI

In [ ]:
# defining bmi and bmi ranges
underweight = df[df['bmi'] <= 18.5]
normal = df[(df['bmi'] > 18.5) & (df['bmi'] <= 24.9)]
overweight = df[(df['bmi'] >= 25) & (df['bmi'] <= 29.9)]
obese = df[df['bmi'] >= 30]

In [ ]:
# seeing general bmi distribution
fig_bmi = go.Figure()
fig_bmi.add_trace(go.Histogram(x = df['bmi'], marker_color = '#2F4F4F'))
fig_bmi.update_layout( title_text='BMI Distribution', xaxis_title_text='BMI', yaxis_title_text='Count', bargap=0.05, template = 'seaborn', width=500, height=600)
fig_bmi.show()

In [ ]:
# visualizing bmi by bmi range
fig_bmi = go.Figure()
fig_bmi.add_trace(go.Bar(x = ['Underweight'], y = [len(underweight)], name = 'Underweight', marker_color = '#87CEFA'))
fig_bmi.add_trace(go.Bar(x = ['Normal'], y = [len(normal)], name = 'Normal', marker_color = '#B0C4DE'))
fig_bmi.add_trace(go.Bar(x = ['Overweight'], y = [len(overweight)], name = 'Overweight', marker_color = '#D3D3D3'))
fig_bmi.add_trace(go.Bar(x = ['Obese'], y = [len(obese)], name = 'Obese', marker_color = '#778899'))
fig_bmi.update_layout( title_text = 'Total Count of BMI ', xaxis_title_text = 'BMI Range', yaxis_title_text = 'Count', bargap = 0.05, template = 'seaborn', width = 500, height = 600
)
fig_bmi.show()

In [ ]:
fig_bmi = go.Figure()
fig_bmi.add_trace(go.Scatter(x = underweight['bmi'], y = underweight['charges'], marker_color = '#87CEFA', mode = 'markers', name = 'Underweight'))
fig_bmi.add_trace(go.Scatter(x = normal['bmi'], y = normal['charges'], marker_color = '#B0C4DE', mode = 'markers', name = 'Normal'))
fig_bmi.add_trace(go.Scatter(x = overweight['bmi'], y = overweight['charges'], marker_color = '#D3D3D3', mode = 'markers', name = 'Overweight'))
fig_bmi.add_trace(go.Scatter(x = obese['bmi'], y = obese['charges'], marker_color = '#778899', mode = 'markers', name = 'Obsese'))
fig_bmi.update_layout( title_text = 'BMI and Medical Charges', xaxis_title_text = 'BMI Range', yaxis_title_text = 'Charges', template = 'seaborn', width = 600, height = 600
)
fig_bmi.show()

In [ ]:
fig_bmi = go.Figure()
fig_bmi.add_trace(go.Box(y = underweight['charges'], boxpoints = 'all', marker_color = '#87CEFA', name = 'Underweight'))
fig_bmi.add_trace(go.Box(y = normal['charges'], boxpoints = 'all', marker_color = '#B0C4DE', name = 'Normal'))
fig_bmi.add_trace(go.Box(y = overweight['charges'], boxpoints = 'all', marker_color = '#D3D3D3', name = 'Overweight'))
fig_bmi.add_trace(go.Box(y = obese['charges'], boxpoints = 'all', marker_color = '#778899', name = 'Obese'))
fig_bmi.update_layout( title_text = 'BMI and Charges', xaxis_title_text = 'BMI Range', yaxis_title_text = 'Charges', template = 'seaborn', width = 600, height = 600
)
fig_bmi.show()

### Children

In [ ]:
# defining children ranges
no_children = df[df['children'] == 0]
one_children = df[df['children'] == 1]
two_children = df[df['children'] == 2]
more_children = df[df['children'] >= 3]

In [ ]:
# visualizing children ranges
fig_children = go.Figure()
fig_children.add_trace(go.Bar(x = ['Zero'], y = [len(no_children)], name = 'Zero'))
fig_children.add_trace(go.Bar(x = ['One'], y = [len(one_children)], name = 'One'))
fig_children.add_trace(go.Bar(x = ['Two'], y = [len(two_children)], name = 'Two'))
fig_children.add_trace(go.Bar(x = ['More'], y = [len(more_children)], name = 'More'))
fig_children.update_layout( title_text = 'Total Count of Children', xaxis_title_text = 'Number Of Children', yaxis_title_text = 'Count', bargap = 0.05, template = 'seaborn', width = 600, height = 600
)
fig_children.show()

In [ ]:
# visualizing boxplot of childrena and cost
fig_children = go.Figure()
fig_children.add_trace(go.Box(y = no_children['charges'], boxpoints = 'all', name = 'Zero'))
fig_children.add_trace(go.Box(y = one_children['charges'], boxpoints = 'all', name = 'One'))
fig_children.add_trace(go.Box(y = two_children['charges'], boxpoints = 'all', name = 'Two'))
fig_children.add_trace(go.Box(y = more_children['charges'], boxpoints = 'all', name = 'More'))
fig_children.update_layout(title_text = 'Number Of Children and Medical Cost', xaxis_title_text = 'Number Of Childrens', yaxis_title_text = 'Medical Cost', template = 'seaborn', width = 600, height = 600
)
fig_children.show()

### Smoker

In [ ]:
# defining smokers
smoker = df[df['smoker'] == 'yes']
no_smoker = df[df['smoker'] == 'no']

In [ ]:
# visualizing smokers with pie chart
colors= ['#ff3e3b', '#3da4ff']
labels = ['Smoker', 'Non Smoker']
values = [len(smoker), len(no_smoker)]
fig_smoker = go.Figure()
fig_smoker.add_trace(go.Pie(labels = labels, values = values, hole = .4, marker_colors = colors))
fig_smoker.update_layout( title_text = 'Total Count of Smoker', template = 'seaborn', width = 500, height = 600)
fig_smoker.show()

In [ ]:
# visualizing boxplot of smoker and charges
fig_smoker = go.Figure()
fig_smoker.add_trace(go.Box(y = smoker['charges'], boxpoints = 'all', marker_color = '#ff3e3b', name = 'Smokers'))
fig_smoker.add_trace(go.Box(y = no_smoker['charges'], boxpoints = 'all', marker_color = '#3da4ff', name = 'Non smokers'))
fig_smoker.update_layout( title_text = 'Smoker and Charges', xaxis_title_text = 'Smoker', yaxis_title_text = 'Charges', template = 'seaborn', width = 600, height = 600)
fig_smoker.show()

### Region

In [ ]:
# defining regions
southwest = df[df['region'] == 'southwest']
southeast = df[df['region'] == 'southeast']
northwest = df[df['region'] == 'northwest']
northeast = df[df['region'] == 'northeast']

In [ ]:
fig_region = go.Figure()
fig_region.add_trace(go.Bar(x = ['Southwest'], y = [len(southwest)], name = 'Southwest', marker_color = '#D3D3D3'))
fig_region.add_trace(go.Bar(x = ['southeast'], y = [len(southeast)], name = 'Southeast', marker_color = '#E0FFFF'))
fig_region.add_trace(go.Bar(x = ['northwest'], y = [len(northwest)], name = 'Northwest', marker_color = '#87CEFA'))
fig_region.add_trace(go.Bar(x = ['northeast'], y = [len(northeast)], name = 'Northeast', marker_color = '#ADD8E6'))
fig_region.update_layout(title_text = 'Total Count of Region ', xaxis_title_text = 'Region', yaxis_title_text = 'Count', bargap = 0.05, template = 'seaborn', width = 500, height = 600)
fig_region.show()

In [ ]:
# visualizing boxplot of region and charges
fig_region = go.Figure()
fig_region.add_trace(go.Box(y = southwest['charges'], marker_color = '#D3D3D3', boxpoints = 'all', name = 'Southwest'))
fig_region.add_trace(go.Box(y = southeast['charges'], marker_color = '#E0FFFF', boxpoints = 'all', name = 'Southeast'))
fig_region.add_trace(go.Box(y = northwest['charges'], marker_color = '#87CEFA', boxpoints = 'all', name = 'Northwest'))
fig_region.add_trace(go.Box(y = northeast['charges'], marker_color = '#ADD8E6', boxpoints = 'all', name = 'Northeast'))
fig_region.update_layout( title_text = 'Region and Charges', xaxis_title_text = 'Region', yaxis_title_text = 'Charges', template = 'seaborn', width = 600, height = 600)
fig_region.show()

## Encoding Categorical Variables
Currently, three columns (sex, smoker, and region) are non-numerical. This will affect the data analysis if ignored. To fix this, we will convert the object data types into categorical, and then use LabelEncoding to turn the categorical types into numerical.

In [ ]:
# changing sex, smoker and region columns to numbers since they are object data type

# sex (female/male)
df['sex'] = df['sex'].astype('category')

# smoker(yes/no)
df['smoker'] = df['smoker'].astype('category')

# region
df['region'] = df['region'].astype('category')

In [ ]:
# using LabelEncoder to turn categorical variables into numerical
le = LabelEncoder ()

# sex
le.fit(df.sex.drop_duplicates())
df.sex = le.transform(df.sex)

# smoker
le.fit(df.smoker.drop_duplicates())
df.smoker = le.transform(df.smoker)

# region
le.fit(df.region.drop_duplicates())
df.region = le.transform(df.region)

In [ ]:
# checking data types once more
df.dtypes

In [ ]:
# seeing what the encoded variables will look like
df.head()

In [ ]:
# exporting encoded data as .csv file
df.to_csv('Encoded Dataset.csv', index = False)

We can now see that all the columns are numerical after encoding the variables. With the encoded variables the, here are the updated values for each column:
* **sex** column has 0 = female and 1 = male
* **smoker** column has 0 = no and 1 = yes
* **region** column has 1 = northwest, 2 = southeast, 3 = southwest, and 4 = northeast

## Visualizations with Encoded Data
We will create visualizations with the encoded data.

In [ ]:
# visualizing a pairplot between age, bmi and charges
sns.pairplot(df[['age', 'bmi', 'charges']])

In [ ]:
# creating correlation heatmap
f, ax = plt.subplots(1, 1, figsize=(10, 10))
ax = sns.heatmap(df.corr(), annot=True, cmap='Blues')

The variable with the highest correlation to charges is smoker, with a correlation of .79. The second most correlated variable is bmi with .16.

In [ ]:
# seeing general distribution of charges
f = plt.figure(figsize = (11,4))
sns.set(style='white')

# without natural log
ax = f.add_subplot(121)
sns.distplot(df['charges'], kde = True, color = 'cornflowerblue', ax = ax)
ax.set_title('Distribution of Charges')

# with natural log
ax = f.add_subplot(122)
sns.distplot(np.log10(df['charges']), kde = True, color = 'slategray', ax = ax)
ax.set_title('Distribution of Charges w/ Natural Log')

The left graph does not look like a normal curve, so we will apply ln (natural log) to normalize the distribution, which is shown on the right.

In [ ]:
# seeing general distribution of charges for smokers and non-smokers
f = plt.figure(figsize = (12, 5))

# non smokers
ax = f.add_subplot(121)
sns.distplot(df[(df.smoker == 0)]['charges'], color = 'darkslategray', ax = ax)
ax.set_title('Distribution of Charges for Non-Smokers')

# smokers
ax = f.add_subplot(122)
sns.distplot(df[(df.smoker == 1)]['charges'], color = 'black', ax = ax)
ax.set_title('Distribution of Charges for Smokers')

The graph for smokers shows a prevalence of higher medical charges compared to non-smokers. The two peaks for smokers are at around 20,000-30,000 dollars and the peak for non-smokers is below 5,000 dollars. 

In [ ]:
# bar plot to visualize smokers by sex for charges
f, ax = plt.subplots(1,1, figsize=(9,8))
ax = sns.barplot(x = 'smoker', y = 'charges',
                 hue='sex', data=df, palette='Blues')

According to this graph, there are more female non-smokers than males, and more male smokers than females. It is still prevalent that smokers have higher medical charges than the non-smokers.

In [ ]:
# boxplots to visualize different variables with charges
plt.figure(figsize=(15, 10))
plt.subplot(2,2,1)
sns.boxplot(x = 'sex', y = 'charges', data = df, palette = 'Greys')
plt.subplot(2,2,2)
sns.boxplot(x = 'children', y = 'charges', data = df, palette = 'Blues')
plt.subplot(2,2,3)
sns.boxplot(x = 'smoker', y = 'charges', data = df, palette = 'bone')
plt.subplot(2,2,4)
sns.boxplot(x = 'region', y = 'charges', data = df, palette = 'Purples')
plt.show()

From these boxplots, there is not much difference between the regions in terms of medical charges. It seems like people who do not have children have higher medical charges than those who do have children. As visualized, non-smokers have lower medical costs compared to smokers.

In [ ]:
# seeing distribution with age and charges
ax = sns.lmplot(x = 'age', y = 'charges', data = df, hue = 'smoker', palette = 'BuPu')
plt.figure(figsize=(9,5))
plt.title('Distribution of Age')
ax = sns.distplot(df['age'], color = 'royalblue')

In [ ]:
# seeing distribution with bmi and charges
ax = sns.lmplot(x = 'bmi', y = 'charges', data = df, hue = 'smoker', palette = 'bone')
plt.figure(figsize = (9,5))
plt.title('Distribution of BMI')
ax = sns.distplot(df['bmi'], color = 'darkslateblue')

It appears that people with high bmi and who smoke have higher medical charges compared to those with lower bmi and who do not smoke.

In [ ]:
# seeing distribution with children and charges
plt.title('Distribution Between Children and Charges')
sns.barplot(x = df['children'], y = df['charges'], palette = 'BuPu')

# bar plot to visualize smokers by children for charges
sns.catplot(x = 'children', kind = 'count', palette = 'PuBu',hue = 'smoker',
            data = df[(df.children > 0)], size = 4)
ax.set_title('Smokers and Non-smokers With Children')

## Splitting Train/Test Data
Breaking the original data into training and testing sets is vital for machine learning. The training dataset is used to make predictions, whereas the testing dataset (the actual data) is used to compare the predictions. In this case, the training dataset will omit the 'charges' section from the original dataset since the goal of this analysis is to predict future medical charges.

In [ ]:
# creating train and test data
x = df.drop(['charges'], axis = 1)
y = df['charges']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state = 0)

## Random Forest
The random forest algorithm and feature importance will be used to identify which customer groups heavily impact medical charges.

In [ ]:
# running the random forest reg algorithm
fr = RandomForestRegressor(n_estimators = 100, criterion = 'mse', random_state = 1, n_jobs = -1)
fr.fit(x_train,y_train)
fr_train_pred = fr.predict(x_train)
fr_test_pred = fr.predict(x_test)
print('MSE train data: %.3f, MSE test data: %.3f' % (
mean_squared_error(y_train,fr_train_pred),
mean_squared_error(y_test,fr_test_pred)))
print('R2 train data: %.3f, R2 test data: %.3f' % (
r2_score(y_train,fr_train_pred),
r2_score(y_test,fr_test_pred)))

In [ ]:
# printing random forest result scores
y_train_pred = fr.predict(x_train)
y_test_pred = fr.predict(x_test)

print(fr.score(x_test,y_test))

In [ ]:
# Printing performance scores of the algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

In [ ]:
# visualizing predicted and actual values
plt.figure(figsize=(9,6))
plt.scatter(fr_train_pred,fr_train_pred - y_train, c = 'steelblue', marker = 'o', s = 35, alpha = 0.5, label = 'Train data')
plt.scatter(fr_test_pred,fr_test_pred - y_test, c = 'dimgrey', marker = 'o', s = 35, alpha = 0.7, label = 'Test data')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.legend(loc = 'lower right')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'black')
plt.show()

In [ ]:
# Printing list of features by importance
print('Ranking of Features by Importance\n')
important = fr.feature_importances_
std = np.std([tree.feature_importances_ for tree in fr.estimators_],axis=0)
indexes = np.argsort(important)[::-1]
variables = ['age', 'sex', 'bmi', 'children','smoker', 'region']
important_list = []
for f in range(x.shape[1]):
    variable = variables[indexes[f]]
    important_list.append(variable)
    print("%d.%s(%f)" % (f + 1, variable, important[indexes[f]]))

In [ ]:
# visualizing the important features from algorithm
plt.figure()
plt.title("Important Features")
plt.bar(important_list, important[indexes],
       color = "royalblue", yerr = std[indexes], align = "center")

According to the feature importance results, smoker is the category that highly affects charges. After that is bmi, and then age.

In [ ]:
# Predicting the medical charges with random forest
y_test_pred = fr.predict(x_test)

In [ ]:
# Comparing actual recorded values with predicted values
dffr = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})
dffr

## Linear Regression

Because the goal of this analysis is to predict numeral values, linear regression will be used to calculate predicted medical charges. Linear regression is the initial algorithm proposed to use when creating this project. We will test if this is the best algorithm to use. Note that following this section is polynomial regression.

In [ ]:
# running the linear reg algorithm
lr = LinearRegression().fit(x_train,y_train)

In [ ]:
# printing result scores
y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)

print(lr.score(x_test,y_test))

A score of .7998 is not phenomenal. It seems like linear regression is not the best algorithm to use to predict medical charges.

In [ ]:
# printing out intercept
print(lr.intercept_)

In [ ]:
# printing coefficients
print(lr.coef_)

In [ ]:
# Printing performance scores of the algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

In [ ]:
# Predicting the medical charges
y_test_pred = lr.predict(x_test)

In [ ]:
# Comparing actual recorded values with predicted values
dflin = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})
dflin

In [ ]:
# visualizing predicted and actual values
plt.figure(figsize=(9,6))
plt.scatter(y_train_pred, y_train_pred - y_train, c = 'steelblue', marker = 'o', s = 35, alpha = 0.5, label = 'Train data')
plt.scatter(y_test_pred,y_test_pred - y_test, c = 'dimgrey', marker = 'o', s = 35, alpha = 0.7, label = 'Test data')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.legend(loc = 'lower right')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'black')
plt.show()

Using linear regression to predict values with customized attributes
lesex = LabelEncoder()
lesmoker = LabelEncoder()
leregion = LabelEncoder()
sc = StandardScaler()

example person michael
michael = [25, 'male', 30.5, 2, 'yes', 'southeast']
print('Michael: ', str(michael))
michael[1] = lesex.transform([michael[1]])[0]
michael[4] = lesmoker.transform([michael[4]])[0]
michael[5] = leregion.transform([michael[5]])[0]
X = sc.transform([michael])
cost_for_michael = lr.predict(X)[0]
print('Cost for Michael: ',cost_for_michael,'\n')

## Polynomial Regression
If data points do not fit a linear regression precisely, it may be ideal to use polynomial regression.<br><br>

Polynomial regression also uses the relationship between variables x and y in order to identify the best way to draw a line through data points.

### Degree 2
We will first use the polynomial algorithm with degree 2.

In [ ]:
# running polynomial regression algorithm with degree 2
pf = PolynomialFeatures (degree = 2)
xpf = pf.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(xpf, y, test_size = 0.2, random_state = 0)
pfreg = LinearRegression()
pfreg.fit(x_train, y_train)

In [ ]:
# printing intercept, coefficients for degree 2 poly. reg.
ytrain_pred = pfreg.predict(x_train)
ytest_pred = pfreg.predict(x_test)
print(pfreg.intercept_)
print(pfreg.coef_)

In [ ]:
# printing score for degree 2 poly. reg.
print(pfreg.score(x_test, y_test))

A score of .8648 is better than the linear regression's score of .7998

In [ ]:
# Printing performance scores of the algorithm for degree 2 poly. reg.
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

In [ ]:
# Predicting the medical charges for degree 2 poly. reg.
y_test_pred = pfreg.predict(x_test)

In [ ]:
# Comparing actual recorded values with predicted values for degree 2 poly. reg.
dfpreg = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})
dfpreg

In [ ]:
# visualizing predicted and actual values for poly reg. degree 2
plt.figure(figsize=(9,6))
plt.scatter(ytrain_pred, ytrain_pred - y_train, c = 'steelblue', marker = 'o', s = 35, alpha = 0.5, label = 'Train data')
plt.scatter(ytest_pred, ytest_pred - y_test, c = 'dimgrey', marker = 'o', s = 35, alpha = 0.7, label = 'Test data')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.legend(loc = 'lower right')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'black')
plt.show()

Graphically, it seems like polynomial regression with degree 2 is more scattered compared to linear regression.

### Degree 3
Now let's try the algorithm with degree 3.

In [ ]:
# running polynomial regression algorithm with degree 3
pf = PolynomialFeatures (degree = 3)
xpf = pf.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(xpf, y, test_size = 0.2, random_state = 0)
pfreg = LinearRegression()
pfreg.fit(x_train, y_train)

In [ ]:
# printing intercept, coefficients for degree 3 poly. reg.
ytrain_pred = pfreg.predict(x_train)
ytest_pred = pfreg.predict(x_test)
print(pfreg.intercept_)
print(pfreg.coef_)

In [ ]:
# printing score for degree 3 poly. reg.
print(pfreg.score(x_test, y_test))

A score of .8740 is better than the degree 2 polynomial regression score of .8648 and linear regression's score of .7998.

In [ ]:
# printing performance scores of the algorithm for degree 3 poly. reg.
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

In [ ]:
# predicting the medical charges for degree 3 poly. reg.
y_test_pred = pfreg.predict(x_test)

In [ ]:
# comparing actual recorded values with predicted values for degree 3 poly. reg.
dfpreg2 = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})
dfpreg2

In [ ]:
# visualizing predicted and actual values for poly reg. degree 3
plt.figure(figsize=(9,6))
plt.scatter(ytrain_pred, ytrain_pred - y_train, c = 'steelblue', marker = 'o', s = 35, alpha = 0.5, label = 'Train data')
plt.scatter(ytest_pred, ytest_pred - y_test, c = 'dimgrey', marker = 'o', s = 35, alpha = 0.7, label = 'Test data')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.legend(loc = 'lower right')
plt.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'black')
plt.show()

Similar to degree 2 polynomial regression graph, results seem more scattered compared to linear regression.

## XGBoost

params = {
        'max_depth': [3, 4, 5], # depth of each tree
        'n_estimaters': [10, 30, 50, 100],
        'learning_rate' :[0.01, 0.03, 0.05]
        }
xgb_model = xgb.XGBRegressor(random_state = 1)
grid_xgb = GridSearchCV(xgb_model,
                    params,
                    cv = 2, 
                    n_jobs = 5, 
                    verbose = True)

grid_xgb.fit(x_train, y_train)
grid_xgb.best_params_

grid_xgb_R2 = grid_xgb.score(x_test,y_test)
grid_xgb_R2

y_pred = grid_xgb.predict(x_test)
y_pred_orig = scaler_y.inverse_transform(y_pred)
y_test_orig = scaler_y.inverse_transform(y_test)

k = x_test.shape[1]
n = len(x_test)
RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_pred_orig)),'.3f'))
MSE = mean_squared_error(y_test_orig, y_pred_orig)
MAE = mean_absolute_error(y_test_orig, y_pred_orig)
r2 = r2_score(y_test_orig, y_pred_orig)
adj_r2 = 1 - (1-r2) * (n-1)/(n-k)
print('RMSE =',RMSE, 
      '\nMSE =',MSE, 
      '\nMAE =',MAE,
      '\nR2=',r2, 
      '\nAdjusted R2 =',adj_r2)
